In [29]:
import os
import requests
import json
import pandas as pd

from dotenv import load_dotenv

load_dotenv()

True

In [17]:
def get_resource_docs(api_version: str):

    obp_host = os.getenv("OBP_BASE_URL")
    resource_doc_path = f"/obp/v2.1.0/resource-docs/{api_version}/obp"
    resource_doc_url = obp_host + resource_doc_path

    response = requests.get(resource_doc_url)

    return response.json()

In [30]:
resource_docs = get_resource_docs('v5.1.0')
with open('resource_docs.json', 'w') as f:
    json.dump(resource_docs, f, indent=3)

In [51]:
resource_docs_df = pd.DataFrame(resource_docs['resource_docs'])
print(resource_docs_df.info())
for column in resource_docs_df.columns:
    print(f"{column} {type(resource_docs_df.iloc[100][column])}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539 entries, 0 to 538
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   operation_id                 539 non-null    object
 1   implemented_by               539 non-null    object
 2   request_verb                 539 non-null    object
 3   request_url                  539 non-null    object
 4   summary                      539 non-null    object
 5   description                  539 non-null    object
 6   description_markdown         539 non-null    object
 7   success_response_body        474 non-null    object
 8   error_response_bodies        539 non-null    object
 9   tags                         539 non-null    object
 10  typed_success_response_body  465 non-null    object
 11  is_featured                  539 non-null    bool  
 12  special_instructions         539 non-null    object
 13  specified_url                539 no

In [65]:
from pydantic import BaseModel, Field
from typing import List, Dict, Any, Optional

class ResourceDoc(BaseModel):
    operation_id: str = Field(
        description="Unique identifier for the API operation. Can be used to create UUIDs and hashes."
    )
    implemented_by: Dict[str, str] = Field(
        description="Dictionary describing which API version and function the endpoint is implemented by"
    )
    request_verb: str = Field(
        description="HTTP verb (e.g., GET, POST) for this request."
    )
    request_url: str = Field(
        description="URL endpoint for the request i.e. the path on the API",
        examples=['/obp/v5.1.0/banks']
    )
    summary: str = Field(
        description="Short summary of the API operation."
    )
    description: str = Field(
        description="Detailed description of the API operation."
    )
    description_markdown: str = Field(
        description="Markdown-formatted description."
    )
    success_response_body: Optional[Dict[str, Any]] = Field(
        description="Dictionary describing the success response body, if any.",
        default={},
    )
    error_response_bodies: List[Any] = Field(
        description="List of potential error response bodies. These are OBP error response codes.",
        examples=[
            [
                'OBP-20001: User not logged in. Authentication is required!',
                'OBP-50000: Unknown Error.'
            ]
        ]
    )
    tags: List[str] = Field(
        description="List of API tags related to the operation. Used for categorization and filtering."
    )
    typed_success_response_body: Optional[Dict[str, Any]] = Field(
        description="Typed structure for the success response body.",
        default={},
    )
    is_featured: bool = Field(
        description="Indicates whether this operation is featured."
    )
    special_instructions: str = Field(
        description="Any special instructions for using the operation."
    )
    specified_url: str = Field(
        description="Explicitly specified URL if different from request_url."
    )
    connector_methods: List[Any] = Field(
        description="List of connector methods involved."
    )
    example_request_body: Optional[Dict[str, Any]] = Field(
        description="Example of the request body.",
        default={},
    )
    typed_request_body: Optional[Dict[str, Any]] = Field(
        description="Typed schema for the request body.",
        default={},
    )
    roles: Optional[List[Any]] = Field(
        description="List of entitlements (or roles) needed to successfully complete this operation",
        default=[],
    )

In [72]:
from pydantic import ValidationError

errors = []
for doc in resource_docs['resource_docs']:
    print(doc['operation_id'])
    try:
        ResourceDoc.model_validate(doc)
    except ValidationError as e:
        print(f'\nerror validating: {e} saving error...')
        validation_error = {
            'operationId': doc['operation_id'],
            'errors': e.errors()
        }

        errors.append(validation_error)
        #print(json.dumps(doc, indent=4))
        continue

OBPv1.3.0-getCards
OBPv3.0.0-dataWarehouseStatistics
OBPv5.1.0-getCustomersForUserIdsOnly
OBPv5.0.0-createBank
OBPv4.0.0-getCustomersAtAnyBank
OBPv5.1.0-createConsumer
OBPv3.0.0-getTransactionsForBankAccount
OBPv5.0.0-getSystemViewsIds
OBPv4.0.0-getCustomersMinimalAtAnyBank
OBPv4.0.0-updateAuthenticationTypeValidation

error validating: 1 validation error for ResourceDoc
example_request_body
  Input should be a valid dictionary [type=dict_type, input_value=['DirectLogin', 'OAuth1.0...DC', 'OAuth2_OIDC_FAPI'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.10/v/dict_type saving error...
OBPv4.0.0-updateMyPersonalUserAttribute
OBPv1.2.1-updateCounterpartyPrivateAlias
OBPv4.0.0-deleteMyApiCollectionEndpointByOperationId

error validating: 1 validation error for ResourceDoc
success_response_body
  Input should be a valid dictionary [type=dict_type, input_value=True, input_type=bool]
    For further information visit https://errors.pydantic.dev/2.10/v/dict_

In [81]:
# this would probably crash if it gets too long

error_log = ""
for e in errors:
    err_string = f"[ValidationError]: {e['operationId']}\n\n" + '\n'.join(json.dumps(n, indent=3) for n in e['errors']) + "\n"+  "---"*40 + "\n\n\n"
    error_log += err_string

with open("resource_docs_validation_errors.txt", "w") as error_file:
    error_file.write(error_log)